In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("hello world!")

hello world!


In [2]:
# !pip install langchain
# !pip install -U langchain-community
# !pip install sentence-transformers
# !pip install faiss-cpu

In [3]:
# !pip install pypdf

### 벡터스토어 맛보기

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# 로컬에서 사용할 임베딩 모델 지정
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 예제 텍스트 데이터
texts = ["Hello world", "FAISS is fast", "Vector search is cool"]

# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_texts(texts, embedding_model)

# 쿼리 검색
query = "FAISS 관련 정보"
query_embedding = embedding_model.embed_query(query)
docs = vectorstore.similarity_search_by_vector(query_embedding, k=2)

for doc in docs:
    print(doc.page_content)


/var/folders/rb/qx02rx056vs8lh6bs_vyt2hw0000gn/T/ipykernel_4342/77266023.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


FAISS is fast
Hello world


### 시스템비계 split해보기

In [56]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 1️⃣ PDF에서 텍스트 추출
pdf_loader = PyPDFLoader("../data/비계즈/시스템 비계 안전작업 지침.pdf")  # 여기에 PDF 파일 경로 입력
documents = pdf_loader.load()


In [57]:
text = ""
for doc in documents:
    text += doc.page_content

# text =  text.replace("""KOSHA GUIDE\nC - 32 - 2020\n""", '')




print(text[:500])

KOSHA GUIDE
C - 32 - 2020
시스템 비계 안전작업 지침
2020. 12
한국산업안전보건공단안전보건기술지침의 개요
○ 작성자 : (사)한국건설안전협회 최순주
○ 제·개정 경과
- 2007년 7월 건설안전분야 제정위원회 심의
- 2007년 11월 총괄제정위원회 심의
- 2011년 12월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)
- 2020년 11월 건설안전분야 표준제정위원회 심의(개정, 법규개정조항반영)
○ 관련규격 및 자료
- Workplace Safety and Health (Scaffolds) Regulations 2011
- KOSHA GUIDE C-26-2017 낙하물 방지망 설치 지침
- KOSHA GUIDE C-29-2017 수직 보호망 설치 지침
- KOSHA GUIDE C-31-2017 추락 방호망 설치 지침
○ 관련법규․규칙․고시 등
- 산업안전보건기준에 관한 규칙 제1편 제7장 비계
- 노동부 고시 제2020-3호(가설공사 표준안전작업


In [58]:
print(text)

KOSHA GUIDE
C - 32 - 2020
시스템 비계 안전작업 지침
2020. 12
한국산업안전보건공단안전보건기술지침의 개요
○ 작성자 : (사)한국건설안전협회 최순주
○ 제·개정 경과
- 2007년 7월 건설안전분야 제정위원회 심의
- 2007년 11월 총괄제정위원회 심의
- 2011년 12월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)
- 2020년 11월 건설안전분야 표준제정위원회 심의(개정, 법규개정조항반영)
○ 관련규격 및 자료
- Workplace Safety and Health (Scaffolds) Regulations 2011
- KOSHA GUIDE C-26-2017 낙하물 방지망 설치 지침
- KOSHA GUIDE C-29-2017 수직 보호망 설치 지침
- KOSHA GUIDE C-31-2017 추락 방호망 설치 지침
○ 관련법규․규칙․고시 등
- 산업안전보건기준에 관한 규칙 제1편 제7장 비계
- 노동부 고시 제2020-3호(가설공사 표준안전작업 지침)
- 고용노동부 고시 제2020-33호(방호장치 안전인증 고시)
- 고용노동부 고시 제2020-53호(사업장 위험성평가에 관한 지침)
○ 기술지침의 적용 및 문의
- 이 기술지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈페이지(www.kosha.or.kr)
안전보건기술지침 소관 분야별 문의처 안내를 참고하시기 바랍니다.
- 동 지침 내에서 인용된 관련 규격 및 자료 등에 관하여 최근 개정 내용이 있는
경우 동 지침에 우선하여 참고하시기 바랍니다.
공표일자 : 2020년 12월
제 정 자 : 한국산업안전보건공단 이사장KOSHA GUIDE
C - 32 - 2020
- 1 -
시스템 비계 안전작업 지침
1. 목 적
이 지침은「산업안전보건기준에 관한 규칙」(이하 “안전보건규칙”이라 한다) 제1편 제
7장 (비계) 및 고용노동부고시 제2020-33호(방호장치 의무안전인증 고시) 규정에 의하
여 시스템 비계를 조립․사용․해체 작업 중 발생할 수 있는 근로자의 떨어짐, 자재에
의한 

In [7]:
import re
sub_text =  re.sub(pattern=r'<그림 \d+>(와|과) 같이', repl='', string=text)
sub_text2 = re.sub(pattern=r"\n<그림 \d+>.*\n", repl="\n", string=sub_text)
sub_text3 = re.sub(pattern=r"- \d+ -", repl='', string=sub_text2)

In [8]:
print(sub_text3)

docs = text_splitter.split_text(sub_text3)

시스템 비계 안전작업 지침
2020. 12
한국산업안전보건공단안전보건기술지침의 개요
○ 작성자 : (사)한국건설안전협회 최순주
○ 제·개정 경과
- 2007년 7월 건설안전분야 제정위원회 심의
- 2007년 11월 총괄제정위원회 심의
- 2011년 12월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)
- 2020년 11월 건설안전분야 표준제정위원회 심의(개정, 법규개정조항반영)
○ 관련규격 및 자료
- Workplace Safety and Health (Scaffolds) Regulations 2011
- KOSHA GUIDE C-26-2017 낙하물 방지망 설치 지침
- KOSHA GUIDE C-29-2017 수직 보호망 설치 지침
- KOSHA GUIDE C-31-2017 추락 방호망 설치 지침
○ 관련법규․규칙․고시 등
- 산업안전보건기준에 관한 규칙 제1편 제7장 비계
- 노동부 고시 제2020-3호(가설공사 표준안전작업 지침)
- 고용노동부 고시 제2020-33호(방호장치 안전인증 고시)
- 고용노동부 고시 제2020-53호(사업장 위험성평가에 관한 지침)
○ 기술지침의 적용 및 문의
- 이 기술지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈페이지(www.kosha.or.kr)
안전보건기술지침 소관 분야별 문의처 안내를 참고하시기 바랍니다.
- 동 지침 내에서 인용된 관련 규격 및 자료 등에 관하여 최근 개정 내용이 있는
경우 동 지침에 우선하여 참고하시기 바랍니다.
공표일자 : 2020년 12월
제 정 자 : 한국산업안전보건공단 이사장
시스템 비계 안전작업 지침
1. 목 적
이 지침은「산업안전보건기준에 관한 규칙」(이하 “안전보건규칙”이라 한다) 제1편 제
7장 (비계) 및 고용노동부고시 제2020-33호(방호장치 의무안전인증 고시) 규정에 의하
여 시스템 비계를 조립․사용․해체 작업 중 발생할 수 있는 근로자의 떨어짐, 자재에
의한 맞음, 감전 등의 재해를 예방함을 목적으로 한다.
2. 적용범위
이 지침은 건설공사 현장에서 시스템 비

In [9]:
for doc in docs:
    print(doc)
    print('=' * 50)

시스템 비계 안전작업 지침

2020. 12
한국산업안전보건공단안전보건기술지침의 개요
○ 작성자 : (사)한국건설안전협회 최순주
○ 제·개정 경과
- 2007년 7월 건설안전분야 제정위원회 심의
- 2007년 11월 총괄제정위원회 심의
- 2011년 12월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)
- 2020년 11월 건설안전분야 표준제정위원회 심의(개정, 법규개정조항반영)
○ 관련규격 및 자료
- Workplace Safety and Health (Scaffolds) Regulations 2011
- KOSHA GUIDE C-26-2017 낙하물 방지망 설치 지침
- KOSHA GUIDE C-29-2017 수직 보호망 설치 지침
- KOSHA GUIDE C-31-2017 추락 방호망 설치 지침
○ 관련법규․규칙․고시 등
- 산업안전보건기준에 관한 규칙 제1편 제7장 비계
- 노동부 고시 제2020-3호(가설공사 표준안전작업 지침)
- 고용노동부 고시 제2020-33호(방호장치 안전인증 고시)
- 고용노동부 고시 제2020-53호(사업장 위험성평가에 관한 지침)
○ 기술지침의 적용 및 문의
- 이 기술지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈페이지(www.kosha.or.kr)
안전보건기술지침 소관 분야별 문의처 안내를 참고하시기 바랍니다.
- 동 지침 내에서 인용된 관련 규격 및 자료 등에 관하여 최근 개정 내용이 있는
경우 동 지침에 우선하여 참고하시기 바랍니다.
공표일자 : 2020년 12월
제 정 자 : 한국산업안전보건공단 이사장
시스템 비계 안전작업 지침
1. 목 적
이 지침은「산업안전보건기준에 관한 규칙」(이하 “안전보건규칙”이라 한다) 제1편 제
7장 (비계) 및 고용노동부고시 제2020-33호(방호장치 의무안전인증 고시) 규정에 의하
여 시스템 비계를 조립․사용․해체 작업 중 발생할 수 있는 근로자의 떨어짐, 자재에
의한 맞음, 감전 등의 재해를 예방함을 목적으로 한다.
2. 적용범위
이 지침은 건설공사 현장에서 시스템 

In [10]:
import re

# text = """
# 1. 첫 번째 항목
# 2. 두 번째 항목
# 3. 세 번째 항목
# 4. 네 번째 항목
# """

pattern = r"(\n\d+\. .*?)(?=\n\d+\. |$)"
matches = re.findall(pattern, sub_text3, re.DOTALL)

for idx, match in enumerate(matches):
    print(f'{idx}번째 text')
    print(match.strip())  # strip()을 사용하여 앞뒤 공백 제거
    print("=" * 60)


0번째 text
2020. 12
한국산업안전보건공단안전보건기술지침의 개요
○ 작성자 : (사)한국건설안전협회 최순주
○ 제·개정 경과
- 2007년 7월 건설안전분야 제정위원회 심의
- 2007년 11월 총괄제정위원회 심의
- 2011년 12월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)
- 2020년 11월 건설안전분야 표준제정위원회 심의(개정, 법규개정조항반영)
○ 관련규격 및 자료
- Workplace Safety and Health (Scaffolds) Regulations 2011
- KOSHA GUIDE C-26-2017 낙하물 방지망 설치 지침
- KOSHA GUIDE C-29-2017 수직 보호망 설치 지침
- KOSHA GUIDE C-31-2017 추락 방호망 설치 지침
○ 관련법규․규칙․고시 등
- 산업안전보건기준에 관한 규칙 제1편 제7장 비계
- 노동부 고시 제2020-3호(가설공사 표준안전작업 지침)
- 고용노동부 고시 제2020-33호(방호장치 안전인증 고시)
- 고용노동부 고시 제2020-53호(사업장 위험성평가에 관한 지침)
○ 기술지침의 적용 및 문의
- 이 기술지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈페이지(www.kosha.or.kr)
안전보건기술지침 소관 분야별 문의처 안내를 참고하시기 바랍니다.
- 동 지침 내에서 인용된 관련 규격 및 자료 등에 관하여 최근 개정 내용이 있는
경우 동 지침에 우선하여 참고하시기 바랍니다.
공표일자 : 2020년 12월
제 정 자 : 한국산업안전보건공단 이사장
시스템 비계 안전작업 지침
1번째 text
1. 목 적
이 지침은「산업안전보건기준에 관한 규칙」(이하 “안전보건규칙”이라 한다) 제1편 제
7장 (비계) 및 고용노동부고시 제2020-33호(방호장치 의무안전인증 고시) 규정에 의하
여 시스템 비계를 조립․사용․해체 작업 중 발생할 수 있는 근로자의 떨어짐, 자재에
의한 맞음, 감전 등의 재해를 예방함을 목적으로 한다.
2번째 text
2. 적용범위
이 지침은 건설공

In [11]:
long_text = matches[4]
long_text

'\n4. 시스템비계 설치․해체 안전작업 절차\n시스템비계 설치․해체 작업은  작업절차를 준수하여야 한다.\n작업절차 준수사항\n4.1 설치계획서 검토 4.1.1 일반안전사항\n↓\n4.2 자재하역 및 운반작업 4.2.1 자재하역 및 운반작업 중 준수사항\n↓\n4.3 설치작업 4.3.1 받침철물(잭베이스), 벽이음 및 가새 설치 준수사항\n4.3.2 수직재 및 수평재 설치 준수사항\n↓\n4.4 안전시설 설치 4.4.1 통로 및 안전방망 설치 준수사항\n↓\n4.5 해체작업 4.5.1 해체작업 시 준수사항\n4.1 설치계획서 사전검토\n4.1.1 일반안전사항\n(1) 시스템비계 설치․해체 작업은 작업단계별 유해·위험요인을 파악하고 해당 유해·\n위험요인을 도출하여 감소대책의 수립 등 위험성평가를 실시하여야 한다.\n(2) 시스템 비계는 작업전 조립도(평면도, 입면도, 단면도, 연결 상세도 등)를 작성하\n고 조립도에 따라 조립하여야 한다.\n(3) 조립도에는 수직재와 수평재 및 벽이음, 승강통로 등의 조립 간격을 표기하여야\n한다.\n(4) 시스템 비계의 높이가 31m이상인 경우에는 (2)에도 불구하고 해당 전문가의 안전\n확인을 받아야한다.\n(5) 해당 전문가의 안전확인을 받은 경우 시스템 비계는 설계 조건과 결과를\n확인(작업조건, 하중조건, 풍하중, 수직보호망 충실률, 벽이음 간격 등)하여 조립\n하고 사용하여야 한다.\n(6) 시스템 비계의 재료는 안전인증제품을 사용하여야 하며, 변형․부식 또는 심하게\n손상된 것을 사용해서는 아니되며 재사용 재료는 품질관리를 하여야 한다.\n(7) 조립ㆍ해체 또는 변경 작업은 작업내용을 보기 쉬운 장소에 게시하고, 작업 구역\n에는 해당 작업에 종사하는 근로자가 아닌 사람의 출입을 금지하여야 한다. 또한,\n신호수를 배치하는 경우 일정한 신호방법을 정하여 신호하도록 하여야 한다. 그\n내용을 보기 쉬운 장소에 게시하여야 한다.\n(8) 시스템비계를 조립․해체 및 변경 작업은 관리감독자의 지휘에 따라 작업하여야\n하

In [12]:
import re

# text = """
# 1.1 첫 번째 주요 항목
# 1.2 두 번째 주요 항목
# 2.1 새로운 섹션의 첫 번째 항목
# 2.2 새로운 섹션의 두 번째 항목
# 3.1 마지막 섹션
# """

pattern = r"(\n\d+\.\d+ .*?)(?=\n\d+\.\d+ |$)"
sub_matches = re.findall(pattern, long_text, re.DOTALL)

# for match in matches:
#     print(match.strip())


for idx, match in enumerate(sub_matches):
    print(f'{idx}번째 text')
    print(match.strip())  # strip()을 사용하여 앞뒤 공백 제거
    print("=" * 60)

0번째 text
4.1 설치계획서 검토 4.1.1 일반안전사항
↓
1번째 text
4.2 자재하역 및 운반작업 4.2.1 자재하역 및 운반작업 중 준수사항
↓
2번째 text
4.3 설치작업 4.3.1 받침철물(잭베이스), 벽이음 및 가새 설치 준수사항
4.3.2 수직재 및 수평재 설치 준수사항
↓
3번째 text
4.4 안전시설 설치 4.4.1 통로 및 안전방망 설치 준수사항
↓
4번째 text
4.5 해체작업 4.5.1 해체작업 시 준수사항
5번째 text
4.1 설치계획서 사전검토
4.1.1 일반안전사항
(1) 시스템비계 설치․해체 작업은 작업단계별 유해·위험요인을 파악하고 해당 유해·
위험요인을 도출하여 감소대책의 수립 등 위험성평가를 실시하여야 한다.
(2) 시스템 비계는 작업전 조립도(평면도, 입면도, 단면도, 연결 상세도 등)를 작성하
고 조립도에 따라 조립하여야 한다.
(3) 조립도에는 수직재와 수평재 및 벽이음, 승강통로 등의 조립 간격을 표기하여야
한다.
(4) 시스템 비계의 높이가 31m이상인 경우에는 (2)에도 불구하고 해당 전문가의 안전
확인을 받아야한다.
(5) 해당 전문가의 안전확인을 받은 경우 시스템 비계는 설계 조건과 결과를
확인(작업조건, 하중조건, 풍하중, 수직보호망 충실률, 벽이음 간격 등)하여 조립
하고 사용하여야 한다.
(6) 시스템 비계의 재료는 안전인증제품을 사용하여야 하며, 변형․부식 또는 심하게
손상된 것을 사용해서는 아니되며 재사용 재료는 품질관리를 하여야 한다.
(7) 조립ㆍ해체 또는 변경 작업은 작업내용을 보기 쉬운 장소에 게시하고, 작업 구역
에는 해당 작업에 종사하는 근로자가 아닌 사람의 출입을 금지하여야 한다. 또한,
신호수를 배치하는 경우 일정한 신호방법을 정하여 신호하도록 하여야 한다. 그
내용을 보기 쉬운 장소에 게시하여야 한다.
(8) 시스템비계를 조립․해체 및 변경 작업은 관리감독자의 지휘에 따라 작업하여야
하며, 관리감독자는 다음의 업무를 수행하여야 한다.
(가) 재료의 결함유무를 점검과 불량

In [13]:
print(sub_text3)

시스템 비계 안전작업 지침
2020. 12
한국산업안전보건공단안전보건기술지침의 개요
○ 작성자 : (사)한국건설안전협회 최순주
○ 제·개정 경과
- 2007년 7월 건설안전분야 제정위원회 심의
- 2007년 11월 총괄제정위원회 심의
- 2011년 12월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)
- 2020년 11월 건설안전분야 표준제정위원회 심의(개정, 법규개정조항반영)
○ 관련규격 및 자료
- Workplace Safety and Health (Scaffolds) Regulations 2011
- KOSHA GUIDE C-26-2017 낙하물 방지망 설치 지침
- KOSHA GUIDE C-29-2017 수직 보호망 설치 지침
- KOSHA GUIDE C-31-2017 추락 방호망 설치 지침
○ 관련법규․규칙․고시 등
- 산업안전보건기준에 관한 규칙 제1편 제7장 비계
- 노동부 고시 제2020-3호(가설공사 표준안전작업 지침)
- 고용노동부 고시 제2020-33호(방호장치 안전인증 고시)
- 고용노동부 고시 제2020-53호(사업장 위험성평가에 관한 지침)
○ 기술지침의 적용 및 문의
- 이 기술지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈페이지(www.kosha.or.kr)
안전보건기술지침 소관 분야별 문의처 안내를 참고하시기 바랍니다.
- 동 지침 내에서 인용된 관련 규격 및 자료 등에 관하여 최근 개정 내용이 있는
경우 동 지침에 우선하여 참고하시기 바랍니다.
공표일자 : 2020년 12월
제 정 자 : 한국산업안전보건공단 이사장
시스템 비계 안전작업 지침
1. 목 적
이 지침은「산업안전보건기준에 관한 규칙」(이하 “안전보건규칙”이라 한다) 제1편 제
7장 (비계) 및 고용노동부고시 제2020-33호(방호장치 의무안전인증 고시) 규정에 의하
여 시스템 비계를 조립․사용․해체 작업 중 발생할 수 있는 근로자의 떨어짐, 자재에
의한 맞음, 감전 등의 재해를 예방함을 목적으로 한다.
2. 적용범위
이 지침은 건설공사 현장에서 시스템 비

In [14]:
long_sub_text =  sub_matches[-3]

In [15]:
pattern = r"(\n\d+\.\d+\.\d+ .*?)(?=\n\d+\.\d+\.\d+ |$)"
sub_match = re.findall(pattern, long_sub_text, re.DOTALL)

# for match in matches:
#     print(match.strip())


for idx, match in enumerate(sub_match):
    print(f'{idx}번째 text')
    print(match.strip())  # strip()을 사용하여 앞뒤 공백 제거
    print("=" * 60)

0번째 text
4.3.1 시스템비계 설치 순서
(가) 편평도 및 다짐 등 지반상태 확인
(나) 받침철물(잭베이스) 설치
(다) 받침철물(잭베이스)에 수직재 설치
(라) 밑둥잡이(수평재) 설치 및 1단부 수평재 설치
(마) 1단부 작업발판 설치
(바) 바닥과 1단부간 가설계단 설치
(사) 1단부 안전난간 및 2단부 수평재 설치
(아) 2단부 작업발판 설치
(자) 1단부와 2단부간 가설계단 설치
(차) 2단부 수직재 설치
(카) 2단부 안전난간 및 3단부 수평재 설치
(타) 벽이음 설치
① 받침철물(잭베이스) 및
수직재 설치
② 밑둥잡이(수평재) 및
1단부 수평재 설치 ③ 1단부 작업발판 설치
④ 1단부 가설계단 설치 ⑤ 1단부 안전난간 및
2단부 수평재 설치 ⑥ 2단부 작업발판 설치
⑦ 2단부 가설계단 설치 ⑧ 2단부 수직재 설치 ⑨ 2단부 안전난간 설치
1번째 text
4.3.2 받침철물(잭베이스), 벽이음 및 가새 설치 준수사항
(1) 지반의 침하방지를 위하여 지내력을 확인하여 연약지반인 경우 <그림 3>과 같
이 깔판․깔목을 설치하거나,  콘크리트를 타설하는 등 침하방
지 조치를 하여야 한다.
(2) 비계 기둥의 밑둥에는 받침철물을 사용하여야 하며, 받침에 고저차가 있는 경우
에는 조절형 받침철물을 사용하여 시스템비계가 항상 수평 및 수직을 유지하도
록 하여야 한다.
(3) 수직재와 받침철물은 밀착되도록 설치하고, 수직재와 받침철물의 연결부 겹침길
이는 받침철물 전체길이의 3분의 1 이상이 되도록 하여야 한다.
(4) 경사진 바닥에 비계기둥을 설치하는 경우에는 피벗형 받침철물 또는 쐐기 등을
사용하여 밑받침 철물의 바닥면이 수평을 유지하도록 하여야 한다.
(5) 벽 연결재 설치간격은  제조사가 정한 기준에 따라 반드시 설
치하여야 한다. 벽 연결재는 수직재와 수평재의 교차부에서 비계 면에 대하여 직
각이 되도록 영구구조물에 연결하여야 한다.
(6) 구조물과 시스템비계 사이에는 추락방호망을 높이 10미터 이내마다 설치하여야 한다.
(7) 가새는 제조사의 매

In [6]:
import re

def spliter(text: str, pattern: str):
    matches = re.findall(pattern, text, re.DOTALL)
    return [match.strip() for match in matches]


def custom_text_splitter(input_text, split_patterns, title_patterns, doc_title):

    text_list = []
    title_list = []
    large_texts =  spliter(input_text, split_patterns[0])
    large_text_titles = spliter(input_text, title_patterns[0])

    for large_text, large_text_title in zip(large_texts, large_text_titles):
        mid_texts = spliter(large_text, split_patterns[1])
        mid_text_titles = spliter(large_text, title_patterns[1])

        if len(mid_texts) == 0:
            title_list.append(large_text_title)
            text_list.append(large_text)

        for mid_text, mid_text_title in zip(mid_texts, mid_text_titles):
            small_texts = spliter(mid_text,split_patterns[2])
            small_text_titles = spliter(mid_text, title_patterns[2])

            if len(small_texts) == 0:
                text_list.append(mid_text)
                title_list.append( large_text_title + '\n' + mid_text_title )
            else :
                text_list += small_texts
                title_list += [large_text_title +'\n' + small_text_title for small_text_title in small_text_titles]

    title_list = [doc_title + ('\n') + title for title in title_list]

    my_text_list = []
    my_title_list = []
    for text, title in zip(text_list, title_list):
        if len(text) > 50:
            my_text_list.append(text)
            my_title_list.append(title)

    return my_title_list, my_text_list

In [39]:
my_title_list, my_text_list =  custom_text_splitter(sub_text3, split_patterns, title_patterns, '시스템 비계 안전작업 지침')

In [42]:
for idx, (text, title) in enumerate( zip(my_text_list, my_title_list)):
    print(f'{idx}번째 텍스트')
    print(f"제목 : {title}")
    print('-' * 60)
    print('내용')
    print(text)
    print("=" * 60)

0번째 텍스트
제목 : 시스템 비계 안전작업 지침
목 적
------------------------------------------------------------
내용
목 적
이 지침은「산업안전보건기준에 관한 규칙」(이하 “안전보건규칙”이라 한다) 제1편 제
7장 (비계) 및 고용노동부고시 제2020-33호(방호장치 의무안전인증 고시) 규정에 의하
여 시스템 비계를 조립․사용․해체 작업 중 발생할 수 있는 근로자의 떨어짐, 자재에
의한 맞음, 감전 등의 재해를 예방함을 목적으로 한다.
1번째 텍스트
제목 : 시스템 비계 안전작업 지침
용어의 정의
------------------------------------------------------------
내용
용어의 정의
(1) 이 지침에서 사용되는 용어의 정의는 다음과 같다
(가) “시스템 비계”라 함은  수직재, 수평재, 가새재 등의 부재를
공장에서 제작하여 현장에서 조립하여 사용하는 가설 구조물을 말한다.
(나) “받침철물”이라 함은 수직재의 하부에 설치하여 미끄러짐이나 침하를 방지하고
비계의 수직과 수평을 유지시키기 위한 조절형 부재를 말한다.
(다) “가새(Bracing)”라 함은 비계에 작용하는 수평방향의 압축․인장력을 지지하는
부재를 말하며 수평재와 수평재, 수직재와 수직재를 경사지게 연결하는 부재
를 말한다.
(라) “수직재”라 함은 상부하중을 하부로 전달하는 시스템비계의 구성하는 부재 중
기둥 부재를 말한다.
(마) “수평재”라 함은 수직재에 직각으로 결합되어 수평하중을 지지하는 부재를 말
한다.
(바) “접합부"라 함은 수직재에 용접으로 고정하여 수직재와 수평재 및 가새재를 조
립할 수 있는 부재(철물)를 말한다.
(사) “벽이음”이라 함은 비계와 건물 등의 구조체에 연결하여 풍하중, 충격하중 등
의 수평 및 수직하중에 의한 인장 및 압축하중을 지지하는 부재를 말한다.
(아) “연결조인트”라 함은 시스템비계의 수직재와 수직재를 상․하로 연결하여 수직
재의 이탈을 방지하기 위하여 사용하는 연

### 두번째꺼 확인해보기!

In [97]:
# 1️⃣ PDF에서 텍스트 추출
pdf_loader = PyPDFLoader("../data/비계즈/교량공사의 이동식 비계공법(MSS) 안전작업 지침.pdf")  # 여기에 PDF 파일 경로 입력
documents = pdf_loader.load()

In [99]:
text = ""
for doc in documents:
    text += doc.page_content

text = re.sub(r'KOSHA GUIDE\nC - \d+ - \d+\n.*\n', repl='\n', string= text)
# text =  text.replace("""KOSHA GUIDE\nC - 32 - 2020\n""", '')
# print(text[:500])

In [105]:
my_title_list, my_text_list =  custom_text_splitter(text, split_patterns, title_patterns, '교량공사의 이동식 비계공법(MSS) 안전작업 지침')

In [108]:
for idx, (text, title) in enumerate( zip(my_text_list, my_title_list)):
    print(f'{idx}번째 텍스트')
    print(f"제목 : {title}")
    print('-' * 60)
    print('내용')
    print(text)
    print("=" * 60)

0번째 텍스트
제목 : 교량공사의 이동식 비계공법(MSS) 안전작업 지침
목 적
------------------------------------------------------------
내용
목 적
이 지침은 산업안전보건기준에 관한 규칙(이하 “안전보건규칙”이라 한다) 제1편
제7장(비계)의 규정에 의하여 강관비계 안전보건작업 관한 기술적 사항을 정함을
목적으로 한다.
1번째 텍스트
제목 : 교량공사의 이동식 비계공법(MSS) 안전작업 지침
용어의 정의
------------------------------------------------------------
내용
용어의 정의
(1) 이 지침에서 사용되는 용어의 정의는 다음과 같다.
(가) “비계”라 함은 공사용 통로나 작업발판 설치를 위하여 구조물의 주위에 조립,
설치되는 가설구조물을 말한다.“
(나) “강관비계”라 함은 비계용 강관을 이음철물(강관조인트 등)이나 조임철물(클램
프 등)을 이용하여 조립한 비계를 말한다.
(다) “비계기둥”이라 함은 비계를 조립할 때 수직으로 세우는 부재를 말한다.
(라) “띠장”이라 함은 비계기둥과 기둥을 직교하여 수평으로 설치하는 부재를 말한다.
(마) “장선”이라 함은 쌍줄비계에서 띠장 사이에 수평으로 걸쳐 작업발판을
지지하는 가로재를 말한다.
(바) “교차가새”라 함은 강관비계 조립시 비계기둥과 띠장을 일체화하고
비계의 도괴에 대한 저항력을 증대시키기 위해 비계 외면에 X형태로 설치하
는 것을 말한다.
(사) “벽이음”이라 함은 강관, 클램프, 앵커 및 벽 연결용 철물 등의 부재를         
사용하여 비계와 구조체 사이를 연결함으로써 풍하중, 충격 등의 수평 및 수
직하중에 대하여 안전하도록 설치하는 부재를 말한다.
(아) “작업발판”이라 함은 비계 등에서 작업자의 통로 및 작업공간으로 사
용되는 발판으로 걸침고리가 용접 또는 리벳 등에 의하여 발판에 일체화되어
제작된 작업대와 걸침고리가 없는 통로용 작업발판 등이 있다.
(자) “강관 비계용 부재”라 함은 비

In [129]:
# 1️⃣ PDF에서 텍스트 추출
path = "../data/비계즈/강관비계 안전작업지침.pdf"

def pdf_spliter(path):
    title = path.split('/')[-1][:-4]
    split_patterns = []

    split_patterns.append(r"\n\d{1,2}\. (.*?)(?=\n\d{1,2}\. |$)")
    split_patterns.append(r"\n\d{1,2}\.\d{1,2} (.*?)(?=\n\d{1,2}\.\d{1,2} |$)")
    split_patterns.append(r"\n\d{1,2}\.\d{1,2}\.\d{1,2} (.*?)(?=\n\d{1,2}\.\d{1,2}\.\d{1,2} |$)")

    title_patterns = []
    title_patterns.append(r"\n\d{1,2}\. (.*?)(?=\n|$)")
    title_patterns.append(r"\n\d{1,2}\.\d{1,2} (.*?)(?=\n|$)")
    title_patterns.append(r"\n\d{1,2}\.\d{1,2}\.\d{1,2} (.*?)(?=\n|$)")

    pdf_loader = PyPDFLoader(path)  # 여기에 PDF 파일 경로 입력
    documents = pdf_loader.load()
    text = ""
    for doc in documents:
        text += doc.page_content

    text = re.sub(r'KOSHA GUIDE\nC - \d+ - \d+\n.*\n', repl='\n', string= text)
    my_title_list, my_text_list =  custom_text_splitter(text, split_patterns, title_patterns, title)

    return my_title_list[2:], my_text_list[2:]

my_title_list, my_text_list = pdf_spliter(path)

In [143]:
pathes = ["../data/비계즈/강관비계 안전작업지침.pdf", "../data/비계즈/시스템 비계 안전작업 지침.pdf"]

my_title_list = []
my_text_list = []
for path in pathes:
    temp_title_list, temp_text_list =  pdf_spliter(path)
    my_title_list += temp_title_list
    my_text_list = temp_text_list
    del temp_title_list, temp_text_list


In [ ]:
import glob
pathes = glob.glob('../data/비계즈/*.pdf')
my_title_list = []
my_text_list = []
for path in pathes:
    temp_title_list, temp_text_list =  pdf_spliter(path)
    my_title_list += temp_title_list
    my_text_list = temp_text_list
    del temp_title_list, temp_text_list

In [141]:
pathes

['../data/비계즈/이동식 비계 설치 및 사용안전 기술지침.pdf',
 '../data/비계즈/교량공사의 이동식 비계공법(MSS) 안전작업 지침.pdf',
 '../data/비계즈/작업의자형 달비계 안전작업 지침.pdf',
 '../data/비계즈/시스템 비계 안전작업 지침.pdf',
 '../data/비계즈/강관비계 안전작업지침.pdf']

In [18]:
from langchain.document_loaders import PyPDFLoader
import re

path =   '../data/비계즈/작업의자형 달비계 안전작업 지침.pdf'
title = path.split('/')[-1][:-4]
split_patterns = []

split_patterns.append(r"\n\d{1,2}\. (.*?)(?=\n\d{1,2}\. |$)")
split_patterns.append(r"\n\d{1,2}\.\d{1,2} (.*?)(?=\n\d{1,2}\.\d{1,2} |$)")
split_patterns.append(r"\n\d{1,2}\.\d{1,2}\.\d{1,2} (.*?)(?=\n\d{1,2}\.\d{1,2}\.\d{1,2} |$)")

title_patterns = []
title_patterns.append(r"\n\d{1,2}\. (.*?)(?=\n|$)")
title_patterns.append(r"\n\d{1,2}\.\d{1,2} (.*?)(?=\n|$)")
title_patterns.append(r"\n\d{1,2}\.\d{1,2}\.\d{1,2} (.*?)(?=\n|$)")

pdf_loader = PyPDFLoader(path)  # 여기에 PDF 파일 경로 입력
documents = pdf_loader.load()
text = ""
for doc in documents:
    text += doc.page_content

text = re.sub(r'KOSHA GUIDE\nC - \d+ - \d+\n.*\n', repl='\n', string= text)
# text = re.sub(r'목\s*차.*\n', repl='',string= text,  flags=re.DOTALL)

my_title_list, my_text_list =  custom_text_splitter(text, split_patterns, title_patterns, title)

for idx, (text, title) in enumerate( zip(my_text_list, my_title_list)):
    print(f'{idx}번째 텍스트')
    print(f"제목 : {title}")
    print('-' * 60)
    print('내용')
    print(text)
    print("=" * 60)

0번째 텍스트
제목 : 작업의자형 달비계 안전작업 지침
목 적 ·······································································································P1
------------------------------------------------------------
내용
목 적 ·······································································································P1
1번째 텍스트
제목 : 작업의자형 달비계 안전작업 지침
적용범위 ·································································································P1
------------------------------------------------------------
내용
적용범위 ·································································································P1
2번째 텍스트
제목 : 작업의자형 달비계 안전작업 지침
용어의 정의 ···························································································P1
------------------------------------------------------------
내용
용어의 정의 ···························································································P1
3번째 텍스트
제목 : 작업의자형 달비계 안전작업 지침
작업의자형 달비계 작업순서 및 구성요소에 대한 안전조치 사항
------------------

In [17]:
my_text_list

[]

In [ ]:
pdf_loader = PyPDFLoader(path)  # 여기에 PDF 파일 경로 입력
documents = pdf_loader.load()
text = ""
for doc in documents:
    text += doc.page_content

# print(text)


'KOSHA GUIDE\nC - 33 - 2022\n작업의자형 달비계 안전작업 지침\n2022. 12.\n한국산업안전보건공단안전보건기술지침의 개요\n◦ 작성자 : 한국안전보건공단 송인용\n개정자 : 한국산업안전보건공단 건설안전실\n◦ 제․개정경과\n- 2007년 10월 건설안전분야 제정위원회 심의\n- 2007년 11월 총괄제정위원회 심의\n- 2011년 12월 건설안전분야 제정위원회 심의(개정)\n- 2022년 2월 건설안전분야 표준제정위원회 심의(개정)\n- 2022년 12월 건설안전분야 표준제정위원회 심의(개정)\n◦ 관련규격 및 자료\n- (KCS 21 60 10 :2020) 비계\n․[자재] 2.5 기타비계-달비계\n․[시공] 3.6 기타비계-달비계\n◦ 관련 법규․규칙․고시 등\n- 산업안전보건기준에 관한 규칙 제35조(관리감독자의 유해·위험 방지 업무 등)\n- 산업안전보건기준에 관한 규칙 제45조(지붕 위에서의 위험 방지)\n- 산업안전보건기준에 관한 규칙 제7장 제4절(달비계, 달대비계 및 걸침비계)\n- 고용노동부고시 제2020-3호 가설공사 표준안전작업지침\n◦ 기술지침의 적용 및 문의\n- 이 기술지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈페이지\n(www.kosha.or.kr) 안전보건기술지침 소관 분야별 문의처 안내를 참고하\n시기 바랍니다.\n- 동 지침 내에서 인용된 관련 규격 및 자료 등에 관하여 최근 개정 내용\n이 있는 경우 동 지침에 우선하여 참고하시기 바랍니다.\n공표일자 : 2022년 12월 31일\n제 정 자 : 한국산업안전보건공단 이사장↳ 산업안전보건기준에 관한 규칙 제35조 및 동 규칙 [별표2] 10.'

### 임베딩된거 시험하기

In [136]:
# 3️⃣ 로컬 임베딩 모델 (Hugging Face)
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

# 4️⃣ FAISS VectorStore에 저장
vectorstore = FAISS.from_texts(my_title_list, embedding_model)


In [140]:
# 5️⃣ 저장된 벡터를 사용해 검색
query = "달비계 작업을 통한 알폼 해체시 상하이동방법에 대한 안전관리 대책 수립 누락"
query_embedding = embedding_model.embed_query(query)
results = vectorstore.similarity_search_by_vector(query_embedding, k=50)

# 6️⃣ 검색된 문서 출력
for idx, result in  enumerate(results):
    print(f'{idx +1}번')
    print(result.page_content)
    print("-" * 30)

1번
작업의자형 달비계 안전작업 지침
에반스 매듭
------------------------------
2번
교량공사의 이동식 비계공법(MSS) 안전작업 지침
설치․이동․해체작업 안전조치사항
비계보 및 거푸집
------------------------------
3번
작업의자형 달비계 안전작업 지침
작업의자형 달비계 작업순서 및 구성요소에 대한 안전조치 사항
매듭작업 시 주의사항
------------------------------
4번
작업의자형 달비계 안전작업 지침
작업의자형 달비계 작업 시 안전준수사항
작업 후 작업의자형 달비계의 보관
------------------------------
5번
작업의자형 달비계 안전작업 지침
피셔맨 매듭
------------------------------
6번
시스템 비계 안전작업 지침
시스템비계 설치․해체 안전작업 절차
해체작업 시 준수사항
------------------------------
7번
작업의자형 달비계 안전작업 지침
관리감독자의 유해·위험 방지 직무수행 사항
------------------------------
8번
작업의자형 달비계 안전작업 지침
작업의자형 달비계 작업 시 안전준수사항
로프 결속작업
------------------------------
9번
시스템 비계 안전작업 지침
시스템비계 설치․해체 안전작업 절차
받침철물(잭베이스), 벽이음 및 가새 설치 준수사항
------------------------------
10번
시스템 비계 안전작업 지침
시스템비계 설치․해체 안전작업 절차
통로 및 안전방망 등 설치 준수사항
------------------------------
11번
작업의자형 달비계 안전작업 지침
작업의자형 달비계 작업 시 안전준수사항
준비작업
------------------------------
12번
교량공사의 이동식 비계공법(MSS) 안전작업 지침
설치․이동․해체작업 안전조치사항
해체작업
------------------------------
13번
작업의자

### 잘 작동하네요!!